In [118]:
import json, re
import nltk
import numpy as np
from sklearn.metrics import accuracy_score
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

filenames = [
    'charliehebdo',
    'ferguson',
    'illary',
    'prince-toronto',
    'sydneysiege',
    'ebola-essien',
    'germanwings-crash',
    'ottawashooting',
    'putinmissing'
]
    

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emiljoswin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [125]:
embedding_dict = {}
n_features = 200

def prepare_embeddings():
    print("Loading Global Vectors....")
    if n_features == 100:
        embeddings_file = 'glove.twitter.27B/glove.twitter.27B.100d.txt'
    else:
        embeddings_file = 'glove.twitter.27B/glove.twitter.27B.200d.txt'

    f = []
    with open(embeddings_file) as file:
        f = file.readlines()

    for fi in f:
        line = fi.split()
        embedding_dict[line[0]] = line[1:] # TODO - convert into float here itself

prepare_embeddings()
print("Completed loading Global Vectors.")

Loading Global Vectors....
Completed loading Global Vectors.


In [126]:


print(len(embedding_dict))
print(embedding_dict['for'])
print('is' in embedding_dict)

1193514
['0.11928', '0.22012', '-0.12575', '-0.071592', '0.036457', '-0.14278', '0.1239', '0.20858', '-0.13643', '-0.58471', '0.33246', '-0.42235', '-0.96011', '-0.2813', '0.036594', '-0.10324', '-0.23895', '0.37437', '0.23307', '0.18362', '-0.31105', '0.32997', '-0.16289', '0.0059497', '0.10002', '0.98538', '0.77692', '0.5707', '0.18615', '0.43621', '0.064573', '0.1086', '-0.086082', '-0.026253', '-0.31894', '-0.53476', '-0.13421', '0.43249', '-0.25937', '0.13175', '-0.27095', '0.22287', '0.15417', '0.067198', '0.067832', '-0.15939', '-0.55298', '-0.15007', '-0.11094', '-0.739', '-0.71714', '-0.18907', '-0.12591', '0.21601', '0.24323', '0.52563', '-0.58062', '0.23899', '0.15097', '0.44639', '0.15569', '0.12019', '0.010435', '-0.20619', '0.19929', '-0.2096', '0.54248', '0.095416', '-0.10734', '-0.32039', '-0.087812', '-0.28497', '0.43925', '0.26048', '0.3382', '0.36391', '0.024349', '0.070522', '0.20768', '0.08272', '-0.13455', '0.47675', '0.7644', '0.77207', '-0.018584', '0.12416', '-

In [127]:
def load_data_from_file(name):
    filename = 'parsed_files/' + name + '.json'
    # print(filename)

    try:
        with open(filename) as f:
            tweets = json.load(f)
    except Exception as e:
        print("Exception")
        print(e.message)

    return tweets

In [128]:
repliesText = []
reply_labels = []
corpus = []

def prepare_data(filenames):
    for name in filenames:
        t = load_data_from_file(name)

        for i in range(len(t[name])):
            thread = t[name][i]
            reply_list = thread['replies']
            for reply in reply_list:
                text = reply["text"]
                label = reply["label"]
                repliesText.append(text)
                reply_labels.append(label)

    # Data Cleaning
    ps = PorterStemmer()
    for oneReply in repliesText:
        cleanText = re.sub('http(\S)+', ' http ', oneReply)
        cleanText = re.sub('@', '', cleanText)
        cleanText = re.sub('#', '', cleanText)
        cleanText = re.sub('\?', ' ? ', cleanText)
        cleanText = re.sub('.', ' . ', cleanText)
        cleanText = re.sub('[0-9]+', ' number ', cleanText)

        cleanText = cleanText.lower()

        splitCleanText = cleanText.split()

        str = ""
        for oneWord in splitCleanText:
#             oneWord = ps.stem(oneWord) # TODO - stemming is a too aggressive here
            str += (' '+ oneWord)
            # str = ' '.join(oneWord)

        corpus.append(str)

prepare_data(filenames)

In [138]:
# X = [[] for i in range(len(corpus))]
X = np.zeros((5046, n_features))
print(X.shape)
print(X[0].shape)
y = reply_labels
# n_features = 100

a = np.array([float(s) for s in embedding_dict['for']]).reshape(n_features, 1)

print(len(corpus))
for i in range(len(corpus)):
    sentence = corpus[i]
    # for sentence in corpus:
    words = sentence.split()
    x_ = np.zeros((n_features, 1))
    l = len(words)
    for word in words:
        if word in embedding_dict:
            feat = np.array([float(s) for s in embedding_dict[word]]).reshape(n_features, 1)
            x_ = np.add(x_, feat)
#             print(feat.shape)
        else:
            print("word not found", word) # NOTHING YES!!
#     x_ = np.divide(x_, l).reshape(n_features,)
    x_ = x_.reshape(n_features,)
    X[i] = x_

(5046, 200)
(200,)
5046


In [139]:
# X = X.reshape(5046, n_features)
# print(X.shape)
# print(y.shape)

# a = np.zeros((3, 2))
# b = np.array([1, 3])
# print(a)
# print(b)
# a[1] = b
# print(a)

In [140]:
from sklearn.model_selection import train_test_split
import sklearn
from collections import Counter

from imblearn.over_sampling import SMOTE

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)
print(X_train.shape)
print(X_train[0].shape)

smote_enn = SMOTE(random_state=2)
X_resampled, y_resampled = smote_enn.fit_sample(X_train, y_train)


from sklearn.ensemble import RandomForestClassifier
num_trees = 4
max_depth = 4
clf = RandomForestClassifier(n_jobs=2, random_state=0, n_estimators=num_trees, max_depth=max_depth)
# clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_resampled, y_resampled)


(3532, 200)
(200,)


/Users/emiljoswin/anaconda3/lib/python3.6/site-packages/imblearn/base.py:306: UserWarning: The target type should be binary.
  warnings.warn('The target type should be binary.')


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=4, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [141]:
y_predict = clf.predict(X_test)

print(sorted(Counter(y_test).items()))
print(sorted(Counter(y_predict).items()))


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predict)
print(cm)
#print(len(reply_labels)
recall = np.diag(cm) / np.sum(cm, axis=1)
precision = np.diag(cm)/ np.sum(cm, axis=0)

f1_score = 2 * (precision*recall)/(precision + recall + 1)
acc = accuracy_score(y_test, y_predict)
print('precision = ',precision)
print('recall = ', recall)
print('f1 score = ',f1_score)
print('RF accuracy = ',acc)

print("\n")

[('comment', 1033), ('deny', 130), ('query', 137), ('support', 214)]
[('comment', 986), ('deny', 528)]
[[673 360   0   0]
 [ 86  44   0   0]
 [ 90  47   0   0]
 [137  77   0   0]]
precision =  [0.68255578 0.08333333        nan        nan]
recall =  [0.65150048 0.33846154 0.         0.        ]
f1 score =  [0.38104088 0.03967538        nan        nan]
RF accuracy =  0.47357992073976224




/Users/emiljoswin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':
